# Recherche d'information - Exemple avec Whoosh
Whoosh est une librairie de recherche d'information qui permet de construire un index inversé et de faire des recherches sur cet index avec des requêtes. À ma connaissance, c'est la seule librairie qui est totalement développée en Python.

Dans cet exemple, nous allons utiliser quelques bulletins de nouvelles pour illustrer les étapes de la création d'un index inversé et de la recherche avec cet index.

Voir documentation https://whoosh.readthedocs.io/en/latest/index.html

## Index inversé - Création de l'index à partir de documents
On commence tout d'abord par créer un répertoire sur votre disque pour stocker l'index inversé.

In [3]:
import os.path

inverted_index_dir = "inverted_index"

if not os.path.exists(inverted_index_dir):
    os.mkdir(inverted_index_dir)

Par la suite, on définit la structure des documents que l'on veut emmagasiner dans l'index. Dans cet exemple, chaque document a 3 champs: un titre, un contenu et un identifiant (path).

Il est important de noter que les champs d'un document ont un type. Par exemple TEXT, NUMERIC, BOOLEAN, DATETIME ou ID (un identifiant). De plus, on peut indiquer au système, avec l'option stored = True, que l'on souhaite stocker les valeurs d'un champs en mémoire, pas seulement les indexer. Dans l'exemple suivant, le champ "content" sera indexé et la chaîne de caractère de ce champs sera également conservée en mémoire. Cela permet de retourner le contenu du document pendant une recherche.

Par la suite, on utilise la fonction create-in pour initialiser l'index inversé. Finalement, on ajoute un certain nombre de documents dans l'index inversé à l'aide d'un index writer. Et on termine le tout avec un commit.

In [4]:
from whoosh.fields import Schema, TEXT, ID
from whoosh import index

schema = Schema(title=TEXT(stored=True), path=ID(stored=True), content=TEXT(stored = True))
ix = index.create_in(inverted_index_dir, schema)

docs = [
       {"title": u"Python set to overtake Java", 
        "content": u"Java's popularity continued to decline this month, almost clearing the path for Python to snatch its spot as the world's second most popular programming language", 
        "path": u"doc1"},
       {"title": u"Apple Confirms Serious New Problems For iPhone Users", 
        "content": u"In iOS 14, iPhone owners have received one of the best generational upgrades in years, but it is far from bug free.", 
        "path": u"doc2"},
       {"title": u"GeForce RTX 3080 and 3090 Shortages", 
        "content": u"If you thought it would become easier to purchase an Nvidia RTX 3080 or 3090 by the end of the year, you might be wrong.", 
        "path": u"doc3"},
       {"title": u"Nvidia publicly apologizes for RTX 3080 launch", 
        "content": u"Nvidia has apologized for the RTX 3080 GPU preorder fiasco, which saw the highly desirable graphics card sold out pretty much everywhere.", 
        "path": u"doc4"},
       {"title": u"Apple Releases tvOS 14.0.2 With Bug Fixes", 
        "content": u"Apple today released tvOS 14.0.2, the second update to the tvOS 14 operating system that was released on September 16.", 
        "path": u"doc5"}
       ]

writer = ix.writer()
for doc in docs:
    writer.add_document(title=doc["title"], content=doc["content"], path=doc["path"]) 
writer.commit()

### Information sur l'index inversé - IndexReader et Postings

Il est possible d'obtenir de l'information sur l'index inversé à l'aide de la classe IndexReader.

In [5]:
import pandas as pd
from whoosh.reading import IndexReader
    
with ix.reader() as reader:
    field = "content"
    docs_path = {docnum: x["path"] for docnum, x in reader.iter_docs()}
    # terms = reader.field_terms(field)
    print("\nNombre de documents dans l'index inversé: ", reader.doc_count())
    print("Les identifiants des documents sont: ", list(docs_path.values()))
    term_features = dict()
    for term in reader.field_terms(field):
        info = reader.term_info(field, term)
        postings = reader.postings(field, term)
        posting_list = [docs_path[x] for x in postings.all_ids()]
        term_features[term] = [info.doc_frequency(), posting_list]
    feature_names = ["Nb de documents", "Liste de postings"]
    df = pd.DataFrame.from_dict(term_features, orient='index', columns=feature_names)
    display(df[31:60])


Nombre de documents dans l'index inversé:  5
Les identifiants des documents sont:  ['doc1', 'doc2', 'doc3', 'doc4', 'doc5']


,Nb de documents,Liste de postings
java,1,[doc1]
language,1,[doc1]
might,1,[doc3]
month,1,[doc1]
most,1,[doc1]
much,1,[doc4]
nvidia,2,"[doc3, doc4]"
one,1,[doc2]
operating,1,[doc5]
out,1,[doc4]


## Recherche - Faire le matching d'une requête avec l'index inversé
Une fois la création de l'index inversé complétée, on peut effectuer une recherche à l'aide d'une requête par mots clés. La séquence de traitement est la suivante:

* la création d'un objet pour effectuer la recherche sur l'index inversé (searcher)
* l'analyse de la requête avec un analyseur (classe QueryParser)
* la recherche de document (fonction search)
* l'affichage des résultats de la recherche.

In [6]:
from whoosh.qparser import QueryParser

def search_content(query, inverted_index):
    with inverted_index.searcher() as searcher:
        field = "content"
        schema = inverted_index.schema
        query_parser = QueryParser(field, schema)
        query = query_parser.parse(query)
        print("REQUÊTE: ", query)
        results = searcher.search(query, terms=True)
        show_hits(results)
        
def show_hits(results): 
    print("\nNOMBRE DE DOCUMENTS RETOURNÉ: ", len(results))
    with ix.searcher() as searcher:
        for hit in results:
            print ("\nDOC: ", hit)
            print ("SCORE: ", hit.score)
            # Was this results object created with terms=True?
            if results.has_matched_terms():
                # What terms matched in the results?
                print("TERMES PRÉSENTS: ", hit.matched_terms())    

Voici un premier exemple de recherche avec les mots clés Nvidia et RTX. Cette recherche retourne les documents doc3 et doc4.



In [7]:
query1 = "Nvidia RTX"
results = search_content(query1, ix)

REQUÊTE:  (content:nvidia AND content:rtx)

NOMBRE DE DOCUMENTS RETOURNÉ:  2

DOC:  <Hit {'content': 'If you thought it would become easier to purchase an Nvidia RTX 3080 or 3090 by the end of the year, you might be wrong.', 'path': 'doc3', 'title': 'GeForce RTX 3080 and 3090 Shortages'}>
SCORE:  3.2578349058378313
TERMES PRÉSENTS:  [('content', b'nvidia'), ('content', b'rtx')]

DOC:  <Hit {'content': 'Nvidia has apologized for the RTX 3080 GPU preorder fiasco, which saw the highly desirable graphics card sold out pretty much everywhere.', 'path': 'doc4', 'title': 'Nvidia publicly apologizes for RTX 3080 launch'}>
SCORE:  2.7252879440636146
TERMES PRÉSENTS:  [('content', b'nvidia'), ('content', b'rtx')]


### À propos de l'analyse de questions...

Essayons maintenant avec 2 autres mots clé: Apple et iPhone.

In [8]:
query = "Apple iPhone"
results = search_content(query, ix)

REQUÊTE:  (content:apple AND content:iphone)

NOMBRE DE DOCUMENTS RETOURNÉ:  0


Aucun résultat! Pourtant Apple et iPhone sont respectivement présents dans le champ content des documents doc5 et doc3.

Il est important de noter ici que, par défaut, le QueryParsing suppose qu'on cherche une conjonction de mots (mot1 AND mot2 AND...). Pour effectuer une recherche partielle (c.-à-d. retrouver des documents contenant un sous-ensemble de mots), on peut ajouter des opérateurs OR dans la formulation de la requête.

In [9]:
query = "Apple OR iPhone"
results = search_content(query, ix)

REQUÊTE:  (content:apple OR content:iphone)

NOMBRE DE DOCUMENTS RETOURNÉ:  2

DOC:  <Hit {'content': 'In iOS 14, iPhone owners have received one of the best generational upgrades in years, but it is far from bug free.', 'path': 'doc2', 'title': 'Apple Confirms Serious New Problems For iPhone Users'}>
SCORE:  2.009965776703248
TERMES PRÉSENTS:  [('content', b'iphone')]

DOC:  <Hit {'content': 'Apple today released tvOS 14.0.2, the second update to the tvOS 14 operating system that was released on September 16.', 'path': 'doc5', 'title': 'Apple Releases tvOS 14.0.2 With Bug Fixes'}>
SCORE:  1.9064185987391422
TERMES PRÉSENTS:  [('content', b'apple')]


Ou bien indiquer au QueryParser de rechercher n'importe lequel des termes dans la requête (option group=qparser.OrGroup). La fonction de recherche devient alors:

In [10]:
from whoosh.qparser import OrGroup, AndGroup

def search_content_or(query, inverted_index):
    with inverted_index.searcher() as searcher:
        field = "content"
        schema = inverted_index.schema
        query_parser = QueryParser(field, schema, group=OrGroup)  # MODIFICATION - peut être passé en argument
        query = query_parser.parse(query)
        print("REQUÊTE: ", query)
        results = searcher.search(query, terms=True)
        show_hits(results)

query = "Apple iPhone"
results = search_content_or(query, ix)

REQUÊTE:  (content:apple OR content:iphone)

NOMBRE DE DOCUMENTS RETOURNÉ:  2

DOC:  <Hit {'content': 'In iOS 14, iPhone owners have received one of the best generational upgrades in years, but it is far from bug free.', 'path': 'doc2', 'title': 'Apple Confirms Serious New Problems For iPhone Users'}>
SCORE:  2.009965776703248
TERMES PRÉSENTS:  [('content', b'iphone')]

DOC:  <Hit {'content': 'Apple today released tvOS 14.0.2, the second update to the tvOS 14 operating system that was released on September 16.', 'path': 'doc5', 'title': 'Apple Releases tvOS 14.0.2 With Bug Fixes'}>
SCORE:  1.9064185987391422
TERMES PRÉSENTS:  [('content', b'apple')]


Il est également possible de rechercher des documents qui ne contiennent pas un ou plusieurs termes spécifiques.



In [11]:
query = "NOT Nvidia"
results = search_content(query, ix)

REQUÊTE:  NOT content:nvidia

NOMBRE DE DOCUMENTS RETOURNÉ:  3

DOC:  <Hit {'content': "Java's popularity continued to decline this month, almost clearing the path for Python to snatch its spot as the world's second most popular programming language", 'path': 'doc1', 'title': 'Python set to overtake Java'}>
SCORE:  1.0
TERMES PRÉSENTS:  []

DOC:  <Hit {'content': 'In iOS 14, iPhone owners have received one of the best generational upgrades in years, but it is far from bug free.', 'path': 'doc2', 'title': 'Apple Confirms Serious New Problems For iPhone Users'}>
SCORE:  1.0
TERMES PRÉSENTS:  []

DOC:  <Hit {'content': 'Apple today released tvOS 14.0.2, the second update to the tvOS 14 operating system that was released on September 16.', 'path': 'doc5', 'title': 'Apple Releases tvOS 14.0.2 With Bug Fixes'}>
SCORE:  1.0
TERMES PRÉSENTS:  []


Ou n'importe quelle combinaison booléenne de mots clé.



In [12]:
query = "iPhone NOT (tvOS OR Apple)"
results = search_content(query, ix)

REQUÊTE:  (content:iphone AND NOT (content:tvos OR content:apple))

NOMBRE DE DOCUMENTS RETOURNÉ:  1

DOC:  <Hit {'content': 'In iOS 14, iPhone owners have received one of the best generational upgrades in years, but it is far from bug free.', 'path': 'doc2', 'title': 'Apple Confirms Serious New Problems For iPhone Users'}>
SCORE:  3.009965776703248
TERMES PRÉSENTS:  [('content', b'iphone')]


### Recherche sur plusieurs champs d'un document

Il est possible de mener une recherche conjointement sur plusieurs champs d'un même document grâce à la classe MultifieldParser.

In [13]:
from whoosh.qparser import MultifieldParser


def search_multifields(query, fields, inverted_index):
    with inverted_index.searcher() as searcher:
        schema = inverted_index.schema
        query_parser = MultifieldParser(fields, schema) 
        query = query_parser.parse(query)
        print("REQUÊTE: ", query)
        results = searcher.search(query, terms=True)
        show_hits(results)

search_multifields("GeForce Nvidia RTX", ["title", "content"], ix)

REQUÊTE:  ((title:geforce OR content:geforce) AND (title:nvidia OR content:nvidia) AND (title:rtx OR content:rtx))

NOMBRE DE DOCUMENTS RETOURNÉ:  1

DOC:  <Hit {'content': 'If you thought it would become easier to purchase an Nvidia RTX 3080 or 3090 by the end of the year, you might be wrong.', 'path': 'doc3', 'title': 'GeForce RTX 3080 and 3090 Shortages'}>
SCORE:  6.842051671668612
TERMES PRÉSENTS:  [('title', b'rtx'), ('content', b'rtx'), ('content', b'nvidia'), ('title', b'geforce')]
